In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
# Load data
df = pd.read_csv('data/raw/Data_Science_Jobs_in_India.csv')
print(f"Original data: {len(df)} rows\n")

# ===== DROP UNNECESSARY COLUMN =====
if 'Unnamed: 0' in df.columns or df.columns[0] == '':
    df = df.drop(df.columns[0], axis=1)
    print("✓ Dropped index column")

# ===== CLEAN COMPANY NAMES =====
df['company_name'] = df['company_name'].str.strip()
print("✓ Company names cleaned")

# ===== CATEGORIZE JOB ROLES =====
def categorize_job_role(title):
    """Categorize jobs into specific roles"""
    if pd.isna(title):
        return 'Other'
    
    title_lower = str(title).lower()
    
    # Data Science roles
    if any(word in title_lower for word in ['data scientist', 'ds ', 'scientist']):
        return 'Data Scientist'
    
    # Data Analyst roles
    elif any(word in title_lower for word in ['data analyst', 'business analyst', 'analyst', 'analytics']):
        return 'Data Analyst'
    
    # Data Engineer roles
    elif any(word in title_lower for word in ['data engineer', 'etl', 'big data', 'engineer']):
        return 'Data Engineer'
    
    # ML Engineer roles
    elif any(word in title_lower for word in ['machine learning', 'ml engineer', 'ai engineer', 'mlops']):
        return 'ML Engineer'
    
    # BI roles
    elif any(word in title_lower for word in ['business intelligence', 'bi ', 'power bi', 'tableau']):
        return 'BI Developer'
    
    # Research roles
    elif any(word in title_lower for word in ['research', 'researcher']):
        return 'Research Scientist'
    
    # Manager roles
    elif any(word in title_lower for word in ['manager', 'lead', 'head', 'director']):
        return 'Manager/Lead'
    
    # Consultant roles
    elif any(word in title_lower for word in ['consultant', 'advisor']):
        return 'Consultant'
    
    else:
        return 'Other Data Role'

df['job_category'] = df['job_title'].apply(categorize_job_role)

print("\n✓ Job categories created:")
print(df['job_category'].value_counts())

# ===== EXTRACT SENIORITY LEVEL =====
def extract_seniority(title):
    """Extract seniority level from job title"""
    if pd.isna(title):
        return 'Mid-Level'
    
    title_lower = str(title).lower()
    
    # Senior level
    if any(word in title_lower for word in ['senior', 'sr.', 'sr ', 'lead', 'principal', 'chief', 'head', 'director', 'manager']):
        return 'Senior'
    
    # Junior level
    elif any(word in title_lower for word in ['junior', 'jr.', 'jr ', 'intern', 'trainee', 'fresher', 'associate', 'entry']):
        return 'Junior'
    
    # Mid level (default)
    else:
        return 'Mid-Level'

df['seniority'] = df['job_title'].apply(extract_seniority)

print("\n✓ Seniority levels:")
print(df['seniority'].value_counts())

# ===== CLEAN SALARY DATA =====
def parse_salary_india(salary_str):
    """Parse Indian salary format (LPA/CTC)"""
    if pd.isna(salary_str):
        return np.nan
    
    # Convert to string and clean
    salary_str = str(salary_str).lower().strip()
    
    # Remove common words
    salary_str = salary_str.replace('lpa', '').replace('ctc', '').replace('₹', '').replace('rs', '').replace(',', '').strip()
    
    # If it's already a number
    try:
        return float(salary_str)
    except:
        pass
    
    # Extract first number found
    numbers = re.findall(r'\d+\.?\d*', salary_str)
    if numbers:
        return float(numbers[0])
    
    return np.nan

# Apply to all salary columns
df['avg_salary_lpa'] = df['avg_salary'].apply(parse_salary_india)
df['min_salary_lpa'] = df['min_salary'].apply(parse_salary_india)
df['max_salary_lpa'] = df['max_salary'].apply(parse_salary_india)

print("\n✓ Salary data cleaned!")
print(f"Jobs with salary data: {df['avg_salary_lpa'].notna().sum()} ({df['avg_salary_lpa'].notna().sum()/len(df)*100:.1f}%)")

# ===== CLEAN EXPERIENCE DATA =====
# min_experience is already numeric, just clean it
df['experience_years'] = df['min_experience'].fillna(0).astype(int)

print(f"\n✓ Experience data cleaned!")
print(f"Experience range: {df['experience_years'].min()} to {df['experience_years'].max()} years")

# ===== EXTRACT SKILLS FROM JOB TITLES =====
# Define key skills to search for in job titles
skills_dict = {
    'python': ['python', 'py'],
    'r': [' r ', ' r,', 'r programming'],
    'sql': ['sql', 'mysql', 'postgresql'],
    'excel': ['excel', 'spreadsheet'],
    'tableau': ['tableau'],
    'power_bi': ['power bi', 'powerbi', 'power-bi'],
    'machine_learning': ['machine learning', 'ml', 'deep learning'],
    'statistics': ['statistics', 'statistical'],
    'spark': ['spark', 'pyspark'],
    'hadoop': ['hadoop'],
    'aws': ['aws', 'amazon web services'],
    'azure': ['azure'],
    'tensorflow': ['tensorflow', 'tf'],
    'pytorch': ['pytorch'],
    'nlp': ['nlp', 'natural language'],
    'computer_vision': ['computer vision', 'cv', 'image processing'],
    'big_data': ['big data', 'bigdata'],
    'etl': ['etl', 'pipeline'],
    'git': ['git', 'github'],
    'docker': ['docker', 'container'],
    'data_visualization': ['visualization', 'viz', 'dashboard']
}

# Search in job titles
df['job_title_lower'] = df['job_title'].fillna('').str.lower()

for skill, keywords in skills_dict.items():
    df[f'skill_{skill}'] = df['job_title_lower'].apply(
        lambda x: 1 if any(keyword in str(x) for keyword in keywords) else 0
    )

df = df.drop('job_title_lower', axis=1)
print("\n✓ Skills extracted from job titles!")

# ===== ADD COUNTRY =====
df['country'] = 'India'

# ===== REMOVE DUPLICATES =====
print(f"\nBefore removing duplicates: {len(df)} rows")
df = df.drop_duplicates(subset=['company_name', 'job_title', 'min_experience'], keep='first')
print(f"After removing duplicates: {len(df)} rows")

# ===== HANDLE OUTLIERS IN SALARY =====
# Remove unrealistic salaries (< 1 LPA or > 100 LPA)
df.loc[df['avg_salary_lpa'] < 1, 'avg_salary_lpa'] = np.nan
df.loc[df['avg_salary_lpa'] > 100, 'avg_salary_lpa'] = np.nan
df.loc[df['min_salary_lpa'] < 1, 'min_salary_lpa'] = np.nan
df.loc[df['max_salary_lpa'] > 100, 'max_salary_lpa'] = np.nan

print("\n✓ Outliers removed (salaries < 1 LPA or > 100 LPA)")

# ===== SELECT FINAL COLUMNS =====
final_columns = [
    'company_name', 'job_title', 'job_category', 'seniority', 'country',
    'experience_years', 'min_experience',
    'avg_salary_lpa', 'min_salary_lpa', 'max_salary_lpa',
    'num_of_salaries'
] + [col for col in df.columns if col.startswith('skill_')]

df_clean = df[final_columns].copy()

# Rename for consistency
df_clean = df_clean.rename(columns={
    'company_name': 'company',
    'experience_years': 'min_experience_clean'
})

# ===== SAVE CLEANED DATA =====
# Create directory if not exists
import os
os.makedirs('data/processed', exist_ok=True)

df_clean.to_csv('data/processed/india_jobs_cleaned.csv', index=False)

print("\n" + "="*60)
print("DATA CLEANING COMPLETE! ✅")
print("="*60)

print(f"\nFinal dataset: {len(df_clean)} rows, {len(df_clean.columns)} columns")

print("\n📊 JOB CATEGORY DISTRIBUTION:")
print(df_clean['job_category'].value_counts())

print("\n📊 SENIORITY DISTRIBUTION:")
print(df_clean['seniority'].value_counts())

print("\n💰 SALARY STATISTICS:")
salary_stats = df_clean['avg_salary_lpa'].describe()
print(f"Median Salary: ₹{df_clean['avg_salary_lpa'].median():.1f} LPA")
print(f"Mean Salary: ₹{df_clean['avg_salary_lpa'].mean():.1f} LPA")
print(f"Min: ₹{df_clean['avg_salary_lpa'].min():.1f} LPA")
print(f"Max: ₹{df_clean['avg_salary_lpa'].max():.1f} LPA")
print(f"Jobs with salary data: {df_clean['avg_salary_lpa'].notna().sum()}")

print("\n🏢 TOP 10 COMPANIES:")
print(df_clean['company'].value_counts().head(10))

print("\n🛠️ TOP SKILLS FOUND:")
skill_cols = [col for col in df_clean.columns if col.startswith('skill_')]
skill_counts = {}
for col in skill_cols:
    skill_name = col.replace('skill_', '').replace('_', ' ').title()
    count = df_clean[col].sum()
    if count > 0:
        skill_counts[skill_name] = count

top_skills = sorted(skill_counts.items(), key=lambda x: x[1], reverse=True)[:10]
for skill, count in top_skills:
    pct = (count / len(df_clean)) * 100
    print(f"  {skill:20s}: {count:4d} jobs ({pct:.1f}%)")

print("\n✓ Cleaned data saved to: data/processed/india_jobs_cleaned.csv")
print("\n🚀 READY FOR ANALYSIS AND DASHBOARD!")

Original data: 1602 rows

✓ Dropped index column
✓ Company names cleaned

✓ Job categories created:
job_category
Data Analyst       749
Data Engineer      430
Data Scientist     373
Other Data Role     50
Name: count, dtype: int64

✓ Seniority levels:
seniority
Mid-Level    860
Senior       742
Name: count, dtype: int64

✓ Salary data cleaned!
Jobs with salary data: 1602 (100.0%)

✓ Experience data cleaned!
Experience range: 0 to 21 years

✓ Skills extracted from job titles!

Before removing duplicates: 1602 rows
After removing duplicates: 1602 rows

✓ Outliers removed (salaries < 1 LPA or > 100 LPA)

DATA CLEANING COMPLETE! ✅

Final dataset: 1602 rows, 32 columns

📊 JOB CATEGORY DISTRIBUTION:
job_category
Data Analyst       749
Data Engineer      430
Data Scientist     373
Other Data Role     50
Name: count, dtype: int64

📊 SENIORITY DISTRIBUTION:
seniority
Mid-Level    860
Senior       742
Name: count, dtype: int64

💰 SALARY STATISTICS:
Median Salary: ₹11.9 LPA
Mean Salary: ₹13.2 LPA